In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker
import matplotlib.dates as mdates
from datetime import datetime, time ,date ,timedelta
from dateutil.relativedelta import relativedelta
import seaborn as sns
pd.options.mode.chained_assignment = None

<h1> Fonctions de base </h1>

In [2]:
"""Fonction pour applatir le résultat d'un groupby sur une composante"""

def flatten (table):
    if type(table.columns)==pd.MultiIndex:
        columns_to_look = [name_tmp for name_tmp in table.columns]

        columns_df = [ str(t[0])+'_'+str(t[1]) for t in columns_to_look]
        columns_df.insert(0,table.index.name)

        df = pd.DataFrame(columns = columns_df)

        index = 0
        for i in table.index:
            row = [table[r][i] for r in columns_to_look]
            row.insert(0,i)
            df.loc[index] = row
            index = index + 1
        return(df)
    else :
        table = pd.DataFrame(table)
        table.reset_index(level=0, inplace=True)
        return table

In [3]:
"""Fonction pour applatir le résultat d'un groupby sur plusieurs composantes"""

def flatten_soft (dataframe):
    res = pd.DataFrame()
    res[dataframe.index.name] = dataframe.index
    for col in dataframe.columns:
        name_tmp=""
        for i in range(len(dataframe.columns[0])):
            name_tmp = name_tmp +'_'+str(col[i])
        res[str(name_tmp)] = dataframe[col].values
    return res

<h1> Fonctions de création de features </h1>

In [6]:
"""Fonction pour attribuer chaque client à une cohorte.
Les paramètres :
- df le dataframe de base commandes à la maille ligne = article commandé
- id_customer le champ d'identifiant client
- id_order le champ d'identifiant commande
- order_date le champ de date de commande
"""

def get_cohort (df, id_customer, id_order, order_date, first_year=None):
    df_cohort = df.sort_values(by=str(order_date),
                               ascending=True).drop_duplicates(subset=str(id_customer), keep='first',inplace=False)
    df_cohort['COHORT'] = df_cohort[str(order_date)].dt.year

    if first_year is not None:
        df_cohort['COHORT'] = df_cohort['COHORT'].apply(lambda x: str('Before '+str(first_year-1)) if x<first_year
                                                       else x)
    df = pd.merge(df, df_cohort[[str(id_customer),'COHORT']], on=str(id_customer), how='outer', suffixes=(False,False))
    return df

In [7]:
"""Fonction pour déterminer le nombre de commandes par client.
Les paramètres :
- df le dataframe de base commandes à la maille ligne = articles
- id_customer le champ d'identifiant client
- id_order le champ d'identifiant commande
"""

def get_order_number_per_customer (df, id_customer, id_order):
    nb_order = flatten(df.groupby([str(id_customer)]).agg({str(id_order) : pd.Series.nunique}))
    nb_order.columns = [str(id_customer),'NB_ORDERS']
    df = pd.merge(df, nb_order[[str(id_customer),'NB_ORDERS']], on=str(id_customer), how='outer', suffixes=(False,False))
    return df                                           

In [2]:
"""Fonction pour déterminer le nombre d'articles par commandes.
Les paramètres :
- df le dataframe de base commandes à la maille ligne = articles
- id_order le champ d'identifiant client
- id_article le champ d'identifiant article
"""

def get_article_number_per_order (df, id_order, id_article):
    nb_article = flatten(df.groupby([str(id_order)]).agg({str(id_article) : pd.Series.nunique}))
    nb_article.columns = [str(id_order),'NB_ARTICLES']
    df = pd.merge(df, nb_article[[str(id_order),'NB_ARTICLES']], on=str(id_order), how='outer', suffixes=(False,False))
    return df  

In [8]:
"""Fonction pour déterminer le numéro des commandes.
Les paramètres :
- df le dataframe de base commandes à la maille ligne = articles
- id_customer le champ d'identifiant client
- id_order le champ d'identifiant commande
"""

def get_order_numero (df, id_customer, id_order, order_date):
    nb_order =  flatten(df.groupby([str(id_order)]).agg({str(id_customer):'last',str(order_date):'last'})).sort_values(by=str(order_date),ascending=True)
    nb_order['ORDER_NUMBER'] = nb_order.groupby([str(id_customer)]).cumcount()+1
    df = pd.merge(df, nb_order[[str(id_order),'ORDER_NUMBER']],on=str(id_order) ,how='left')
    return df        


In [1]:
"""Fonction pour déterminer l'univers majoritaire (l'univers désignant une catégorie de produits plus macro que la 
maille produits) d'une commande.

Les paramètres :
- df le dataframe de base commandes à la maille ligne = articles
- id_order le champ d'identifiant commande
- id_universe le champ décrivant la catégorie d'un article 
- value_article le champ de valeur de l'article (en €)
"""

def get_order_universe (df, id_order, order_date):  
    order_universe = df.groupby([str(id_order),str(id_universe)]).agg({str(value_article):sum}).reset_index().sort_values(by=str(value_article),ascending=False)
    order_universe = order_universe.drop_duplicates(subset=[str(id_order)],keep='first')
    order_universe.columns = [str(id_order),'MAIN_UNIVERSE',str(value_order)]
    df = pd.merge(df,order_universe[[str(id_order),'MAIN_UNIVERSE']],on=str(id_order),how='left',suffixes=(False,False))
    return df    

In [3]:
"""Fonction pour attribuer chaque client une date de première commande.
Les paramètres :
- df le dataframe de base commandes à la maille ligne = article commandé
- id_customer le champ d'identifiant client
- id_order le champ d'identifiant commande
- order_date le champ de date de commande
"""

def get_first_order (df, id_customer, id_order, order_date):
    df_cohort = df.sort_values(by=str(order_date),
                               ascending=True).drop_duplicates(subset=str(id_customer), keep='first',inplace=False)
    order_universe.columns = [str(id_customer),'1ST_ORDER_DATE']
    df = pd.merge(df, df_cohort[[str(order_date),'1ST_ORDER_DATE']], on=str(id_customer), how='outer', suffixes=(False,False))
    return df

In [4]:
"""Fonction pour obtenir la séniorité en base d'un client.
Les paramètres :
- df le dataframe de base commandes à la maille ligne = article commandé
- id_first_order le champs donnant la date de premiere commande d'un client ui peut être obtenu par get_first_order

"""

def get_seniority (df, id_customer, id_first_order):
    df['SENIORITY'] = (datetime.now() - df[str(id_first_order)]).dt.days
    return df

In [6]:
"""Fonction pour déterminer calculer la LTV-X mois d'un client.

Les paramètres :
- df le dataframe de base commandes à la maille ligne = articles
- id_customer le champ d'identifiant client
- id_order le champ d'identifiant commande
- id_first_order le champs donnant la date de premiere commande d'un client ui peut être obtenu par get_first_order
- seniority le champs donnant la séniorité en jour du client dans la base
- value_article le champ de valeur de l'article (en €)
- time_delta la durée sur laquelle on veut calculer la LTV (en nombre de jours)


On renvoit un dataset avec la LTV par client qui peut être ensuite être mergé sur le dataset original pour segmenter la LTV
selon des critères prédéfinis.
"""

def get_LTV (df, id_customer, id_order, id_first_order, seniority, value_article, time_delta):
    df['LTV_DATE'] = df[str(id_first_order)] + timedelta(days = time_delta)
    df_ltv = df.loc[( df[str(id_order)] <= df['LTV_DATE'] ) & ( df[str(seniority)] > time_delta)]
    df_ltv = flatten(df_ltv.groupby([str(id_customer)]).agg({str(value_article):sum}))
    df_ltv.columns = [str(id_customer),'LTV']
    return df_ltv

<h1> Fonctions d'études </h1>

<h4> Analyse cohortes</h4>

Construction du dataset pour afficher les cohortes:

In [ ]:
"""Fonction pour extraire la donnée en cohortes en valeur et volume.

Les paramètres :
- df le dataframe de base commandes à la maille ligne = article dans la commande 
- id_customer le champ d'identifiant client
- id_order le champ d'identifiant commande
- order_date le champ de date de commande
- value_article le champ de valeur de l'article (en €)
- id_cohort le champ de cohorte qui peut être obtenu par get_cohort


La sortie:
- Une fois groupé par cohortes, les colonnes ACTIVE_XXXX permettent d'obtenir les cohortes en volume 
- Une fois groupé par cohortes, les colonnes SALES_XXXX permettent d'obtenir les cohortes en valeur 
"""

def build_cohort(df, id_order, order_date):
    output = pd.DataFrame(columns=[str(id_customer), str(id_cohort)])
    
    df['YEAR_ORDER'] = df[str(order_date)].dt.year
    
    for year in df['YEAR_ORDER'].unique():
        df_tmp_peryear = df.loc[df.YEAR_ORDER == year]
        df_tmp_peryear = flatten(df_tmp_peryear.groupby([str(id_customer)]).agg({str(value_article): sum, str(id_cohort):'last'})
        df_tmp_peryear.columns = [str(id_customer),str('SALES_'+str(year)),str('COHORT_'+str(year))]
        df_tmp_peryear[str('ACTIVE_'+str(year))] = 1
            
        output = pd.merge(output,df_tmp_peryear, on=str(id_customer), how='outer', suffixes=(False,False))
        output[str(id_cohort)] = output[str(id_cohort)].combine_first(output[str('COHORT_'+str(year))])
        output.drop(columns=[str('COHORT_'+str(year))], inplace=True)
    
    return output    

<h4> Analyse bridge</h4>

Construction du waterfall pour segmenter la croissance d'une année vers l'autre en new business, churn, LFL, et reactivation:

In [ ]:
"""Fonction pour construire le waterfall.
Les paramètres :
- df le dataframe de base commandes à la maille ligne = article dans la commande 
- id_customer le champ d'identifiant client
- id_order le champ d'identifiant commande
- order_date le champ de date de commande
- value_article le champ de valeur de la commande (en €)
- id_cohort le champ de cohorte qui peut être obtenu par get_cohort

La sortie:
- Une fois sommé sur l'axe=0, les colonnes NEW_BUSINESS_XXXX, LFL_XXXX, CHURN_XXXX et REACTIVATED_XXXX permettent de tracer
le waterfall
"""

def build_waterfall(df, id_customer, order_date, value_order, id_cohort):
    output = pd.DataFrame(columns=[str(id_customer), str(id_cohort)])
    
    df['YEAR_ORDER'] = df[str(order_date)].dt.year
    
    for year in df['YEAR_ORDER'].unique():
        df_tmp_peryear = df.loc[df.YEAR_ORDER == year]
        df_tmp_peryear = flatten(df_tmp_peryear.groupby([str(id_customer)]).agg({str(value_article): sum, str(id_cohort):'last'})
        df_tmp_peryear.columns = [str(id_customer),str('SALES_'+str(year)),str('COHORT_'+str(year))]
        df_tmp_peryear[str('ACTIVE_'+str(year))] = 1
            
        output = pd.merge(output,df_tmp_peryear, on=str(id_customer), how='outer', suffixes=(False,False))
        output[str(id_cohort)] = output[str(id_cohort)].combine_first(output[str('COHORT_'+str(year))])
        output.drop(columns=[str('COHORT_'+str(year))], inplace=True)
    

    for year in range(int(min(df[str(id_cohort)].unique()))+1,int(max(df[str(id_cohort)].unique()))):
        output[str('NEW_BUSINESS_'+str(int(year+1)))] = output.apply(lambda row: row[str('SALES_'+str(year+1))] 
                                                          if ((row[str('ACTIVE_'+str(year))] == 0) and (row[str('ACTIVE_'+str(year+1))] == 1) and 
                                                             row[str(id_cohort)]==year+1)
                                                          else 0 ,axis=1)
        output[str('LFL_'+str(int(year)))] = output.apply(lambda row: row[str('SALES_'+str(year+1))]-row[str('SALES_'+str(year))]
                                                          if ((row[str('ACTIVE_'+str(year))] == 1) and (row[str('ACTIVE_'+str(year+1))] == 1))
                                                          else 0 ,axis=1)
        output[str('CHURN_'+str(int(year+1)))] = output.apply(lambda row: -row[str('SALES_'+str(year))] 
                                                          if ((row[str('ACTIVE_'+str(year))] == 1) and (row[str('ACTIVE_'+str(year+1))] == 0))
                                                          else 0 ,axis=1)
        output[str('REACTIVATED_'+str(int(year+1)))] = output.apply(lambda row: row[str('SALES_'+str(year+1))] 
                                                          if ((row[str('ACTIVE_'+str(year))] == 0) and (row[str('ACTIVE_'+str(year+1))] == 1) 
                                                              and (row[str(id_cohort)]<year+1))
                                                          else 0 ,axis=1)
                                 
                                 
    output.drop(columns=[str(id_customer), str(id_cohort)], inplace=True)
    output = output.sum(axis=0)      
    return output

<h4> Analyse Sankey</h4>

Construction d'un Sankey par univers de commmande :

In [ ]:
"""Fonction pour construire le waterfall.
Les paramètres :
- df le dataframe de base commandes à la maille ligne = article dans la commande
- id_cohort le champ de cohorte qui peut être obtenu par get_cohort

"""

def build_sankey(df, id_cohort, zoom_cohort=None ):
    if first_year is not None:
        df_sankey =  df.loc[df_sankey[str(id_cohort)] == zoom_year]
    
    